In [ ]:
import sys
sys.path.append("self_reflection")

from dotenv import load_dotenv
load_dotenv(".env")

from typing import List, Tuple, Union

import os
import json

import torch
from datasets import Dataset, load_dataset
from langchain_google_genai import ChatGoogleGenerativeAI
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import FactualCorrectness
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm

from CTRLEval.ctrleval import CTRLEval
from evaluate.loop_eval_utils import evaluate_knowledge, evaluate_response
from evaluate.sent_similarity import Sent_Similar

In [ ]:
ds_name = "hotpotqa"
#ds_name = "pubmedqa"
#ds_name = "delucionqa"
ds = load_dataset("rungalileo/ragbench", ds_name, split="test")
print(len(ds))

In [ ]:
ctrleval_scorer = CTRLEval(
    iwf_dir="self_reflection/CTRLEval/iwf_full.txt",
    prompt_dir="self_reflection/CTRLEval/prompt/prompt_topic.txt",
    verbal_dir="self_reflection/CTRLEval/prompt/verbal_topic.txt",
    device='cuda',
)

# Error: Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
# Ignore because Pegasus uses static, sinusoidal position embeddings (rather than learned embeddings) for both encoder and decoder.

entailment_scorer = Sent_Similar()

In [ ]:
MODEL_NAME = "Qwen/Qwen3-4B-Instruct-2507"
#MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="cuda", dtype=torch.bfloat16).eval()
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
class Args:
    continue_generate = False
    no_number = False
    no_aspect = False

    max_loop = 1
    max_knowledge_loop = 1
    max_response_loop = 1
    demo_num = 0

    threshold_entailment = 0.8
    threshold_fact = -1
    threshold_consistency = -5

    max_sample = 3000
    temperature = 1.0
    top_p = 0.90
    top_k = 50
    num_beams = 1
    max_new_tokens = 128
    repetition_penalty = 1.0

In [ ]:
args = Args()
args.max_loop = 3
args.max_knowledge_loop = 3
args.max_response_loop = 3
args.demo_num = 0
args.threshold_entailment = 0.8
args.threshold_fact = -1.0
args.threshold_consistency = -5
args.max_new_tokens = 512
args.repetition_penalty = 1.1

In [ ]:
def generate_step(
    model: AutoModelForCausalLM,
    tokenizer: AutoTokenizer,
    messages: List[dict[str, str]],
    max_new_tokens: int = 512,
    temperature: float = 0.2,
    top_p: float = 0.9,
    top_k: int = 50,
    num_beams: int = 1,
    repetition_penalty: float = 1.1,
) -> str:
    formatted_chat = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(formatted_chat, return_tensors="pt", add_special_tokens=False).to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=temperature > 0,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            num_beams=num_beams,
            repetition_penalty=repetition_penalty,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    return tokenizer.decode(outputs[0][inputs["input_ids"].size(1) :], skip_special_tokens=True)


def knowledge_loop(
    args: Args,
    question: str,
) -> Tuple[str, List[Tuple[int, str, float]]]:
    print("knowledge_loop")

    THRESHOLD_FACTUAL = args.threshold_fact
    MAX_KNOWLEDGE_LOOP = args.max_knowledge_loop

    candidates = []
    history = []

    messages = [
        {
            "role": "system",
            "content": """You are an AI language model designed to provide accurate, relevant, and comprehensive background knowledge based on the given question.""",
        },
        {
            "role": "user",
            "content": f"Provide background knowledge in one paragraph (approximately 400 words) to answer the question: \"{question}\".",
        },
    ]
    knowledge = generate_step(
        model, 
        tokenizer, 
        messages,
        max_new_tokens=args.max_new_tokens,
        temperature=args.temperature,
        top_p=args.top_p,
        top_k=args.top_k,
        num_beams=args.num_beams,
        repetition_penalty=args.repetition_penalty,
    )

    loop_i = 0
    if MAX_KNOWLEDGE_LOOP > 1:
        factuality_score = evaluate_knowledge(model, args.demo_num, question, knowledge, tokenizer)
        candidates.append([factuality_score, knowledge])
        history.append([loop_i, knowledge, factuality_score])

    loop_i += 1
    while (loop_i < MAX_KNOWLEDGE_LOOP) and factuality_score < THRESHOLD_FACTUAL:
        if args.no_aspect:
            instruction = f"Please refine the knowledge."
        elif args.no_number:
            instruction = f"The knowledge is not strongly supported by empirical evidence. Please refine the knowledge to improve its factuality."
        else:
            instruction = f"The factuality score for the knowledge is {factuality_score} less than {THRESHOLD_FACTUAL}, which means the knowledge is not strongly supported by empirical evidence. Please refine the knowledge to improve its factuality."

        messages = [
            {
                "role": "system",
                "content": "You are an AI language model designed to provide accurate, relevant, and comprehensive background knowledge based on the given question.",
            },
            {
                "role": "user",
                "content": f'The provided background knowledge for the question: "{question}" is "{knowledge}".\n\n{instruction}\nThe refined knowledge should be in one paragraph (approximately 400 words).',
            },
        ]
        knowledge = generate_step(
            model, 
            tokenizer, 
            messages,
            max_new_tokens=args.max_new_tokens,
            temperature=args.temperature,
            top_p=args.top_p,
            top_k=args.top_k,
            num_beams=args.num_beams,
            repetition_penalty=args.repetition_penalty,
        )

        factuality_score = evaluate_knowledge(model, args.demo_num, question, knowledge, tokenizer)

        candidates.append([factuality_score, knowledge])
        history.append([loop_i, knowledge, factuality_score])
        loop_i += 1

    if (MAX_KNOWLEDGE_LOOP > 1) and factuality_score < THRESHOLD_FACTUAL:
        candidates.sort()
        return candidates[-1][-1], history
    else:
        return knowledge, history


def response_loop(
    args: Args,
    question: str,
    final_knowledge: str,
) -> Tuple[str, List[Tuple[int, str, float, float]], float]:
    print("response_loop")

    THRESHOLD_CONS = args.threshold_consistency
    MAX_RESPONSE_LOOP = args.max_response_loop

    candidates = []
    entailment_score_question_list = []
    history = []

    messages = [
        {
            "role": "system",
            "content": "You are an AI language model designed to provide accurate, relevant, and comprehensive answers to questions based on the given background knowledge.",
        },
        {
            "role": "user",
            "content": f"Refer to the background knowledge: \"{final_knowledge}\" and answer the question: \"{question}\" in one paragraph.",
        },
    ]
    response = generate_step(
        model, 
        tokenizer, 
        messages,
        max_new_tokens=args.max_new_tokens,
        temperature=args.temperature,
        top_p=args.top_p,
        top_k=args.top_k,
        num_beams=args.num_beams,
        repetition_penalty=args.repetition_penalty,
    )

    loop_i = 0
    if MAX_RESPONSE_LOOP > 1:
        entailment_score_question, cons_score_knowledge = evaluate_response(entailment_scorer, ctrleval_scorer, question, response, final_knowledge)
        candidates.append([(entailment_score_question + cons_score_knowledge) / 2, response])
        entailment_score_question_list.append(entailment_score_question)
        history.append([loop_i, response, entailment_score_question, cons_score_knowledge])

    loop_i += 1
    while loop_i < MAX_RESPONSE_LOOP and cons_score_knowledge < THRESHOLD_CONS:
        if args.no_aspect:
            instruction = f"Please refine the response."
        elif args.no_number:
            instruction = f"The alignment and consistency between response and knowledge are low. Please refine the response to improve its consistency."
        else:
            instruction = f"The consistency score for the response is {cons_score_knowledge} less than {THRESHOLD_CONS}, which means the alignment and consistency between response and knowledge are low. Please refine the response to improve its consistency."

        messages = [
            {
                "role": "system",
                "content": "You are an AI language model designed to provide accurate, relevant, and comprehensive answers to questions based on the given background knowledge.",
            },
            {
                "role": "user", 
                "content": f"The generated response for the question: \"{question}\" is \"{response}\" based on the background knowledge: \"{final_knowledge}\".\n\n{instruction}\nThe refined response should be in one paragraph.",
            },
        ]
        response = generate_step(
            model, 
            tokenizer, 
            messages,
            max_new_tokens=args.max_new_tokens,
            temperature=args.temperature,
            top_p=args.top_p,
            top_k=args.top_k,
            num_beams=args.num_beams,
            repetition_penalty=args.repetition_penalty,
        )

        entailment_score_question, cons_score_knowledge = evaluate_response(entailment_scorer, ctrleval_scorer, question, response, final_knowledge)
        candidates.append([(entailment_score_question + cons_score_knowledge) / 2, response])
        entailment_score_question_list.append(entailment_score_question)
        history.append([loop_i, response, entailment_score_question, cons_score_knowledge])

        loop_i += 1

    if MAX_RESPONSE_LOOP > 1 and cons_score_knowledge < THRESHOLD_CONS:
        merge = zip(candidates, entailment_score_question_list)
        merge = sorted(merge)
        candidates, entailment_score_question_list = zip(*merge)
        return candidates[-1][-1], history, entailment_score_question_list[-1]
    else:
        return response, history, entailment_score_question


def reflection_loop(
    args: Args,
    question: str,
) -> Tuple[str, str, List[Tuple[int, str, float]], List[Tuple[int, str, float, float]]]:
    all_history_knowledge, all_history_response = [], []

    THRESHOLD_ENTAIL = args.threshold_entailment
    MAX_LOOP = args.max_loop

    candidates = []
    main_loop_i = 0
    print(f"main_loop {main_loop_i}")

    final_knowledge, history_knowledge = knowledge_loop(args, question)
    all_history_knowledge += history_knowledge

    final_response, history_response, entailment_score_question = response_loop(args, question, final_knowledge)
    all_history_response += history_response
    candidates.append([entailment_score_question, final_knowledge, final_response])

    main_loop_i += 1
    while main_loop_i < MAX_LOOP and entailment_score_question < THRESHOLD_ENTAIL:
        print(f"main_loop {main_loop_i}")
        final_knowledge, history_knowledge = knowledge_loop(args, question)
        all_history_knowledge += history_knowledge

        final_response, history_response, entailment_score_question = response_loop(args, question, final_knowledge)
        all_history_response += history_response
        candidates.append([entailment_score_question, final_knowledge, final_response])
        main_loop_i += 1

    if (MAX_LOOP > 1) and entailment_score_question < THRESHOLD_ENTAIL:
        candidates.sort()
        final_knowledge, final_response = candidates[-1][1:]

    return final_knowledge, final_response, all_history_knowledge, all_history_response


In [ ]:
num_samples = 100
dataset = []
for d in tqdm(ds.select(range(num_samples)), total=num_samples):
    question = d["question"]
    reference = d["response"]

    _, final_response, _, _ = reflection_loop(args, question)

    dataset.append(
        {
            "user_input": question,
            "response": final_response,
            "reference": reference,
        }
    )

output_dir = "results/exp-0"
with open(os.path.join(output_dir, f"{MODEL_NAME.replace('/', '~')}_{ds_name}_responses_self-reflection.json"), "w") as f:
    json.dump(dataset, f, indent=2)

In [ ]:
evaluator_llm = LangchainLLMWrapper(
    ChatGoogleGenerativeAI(
        model="gemini-2.5-flash-lite",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=4,
    )
)

In [ ]:
evaluation_dataset = Dataset.from_list(dataset)

ragas_result = evaluate(evaluation_dataset, metrics=[FactualCorrectness()], llm=evaluator_llm)
print(ragas_result)

In [ ]:
ragas_result_df = ragas_result.to_pandas()
ragas_result_df.to_csv(os.path.join(output_dir, f"{MODEL_NAME.replace('/', '~')}_{ds_name}_ragas-results_self-reflection.csv"), index=False)
#ragas_result_df.head()